This script mines tweets to train the model.

Use my followers to get tweets, then say similar stuff.

In [1]:
#retweets tweets about the housing crisis

# Import Tweepy, sleep, credentials.py
import tweepy
import time
from datetime import datetime, timezone, timedelta

import sys, os

from authenticate import *

api = get_authenticated_api()

should have already got credentials


## Identify users to RT

In [2]:
starting_user_set = api.friends(count=200)

We could prune these people in a variety of ways. It might help to get a subset where some of them follow each other, then get the users that _they_ follow together and add them to the list. But this might be needlessly complex, so to start, let's just see what we can do with this group.

## Identify tweets from them that we actually want to use

We want:

 - Retweets only - MAYBE, maybe not. I am not sure. It might be OK to just generate random pro-housing slogans.
 - Tweets specific to housing. I would rather avoid a direct keyword approach. Something social, like where they're retweeting each other, might work.
 
Other options:

 - Get those users' favorites using API.favorites

### Step 1. Get some users' tweets

In [25]:
user_statuses=[]
page_count=0
for status_page in tweepy.Cursor(
        api.user_timeline, screen_name="curiouskiwicat", include_rts=False, 
        tweet_mode='extended', count=200).pages():
    page_count += 1
    print(f'Getting page {page_count} for statuses')
    print(len(status_page))
    user_statuses.extend(status_page)
print(f"number of statuses for user: {len(user_statuses)}")
curiouskiwicat_statuses=user_statuses

Getting page 1 for statuses
154
Getting page 2 for statuses
158
Getting page 3 for statuses
162
Getting page 4 for statuses
145
Getting page 5 for statuses
122
Getting page 6 for statuses
124
Getting page 7 for statuses
138
Getting page 8 for statuses
127
Getting page 9 for statuses
123
Getting page 10 for statuses
147
Getting page 11 for statuses
145
Getting page 12 for statuses
141
Getting page 13 for statuses
150
Getting page 14 for statuses
127
Getting page 15 for statuses
103
Getting page 16 for statuses
124
Getting page 17 for statuses
35
number of statuses for user: 2225


In [39]:
[s.created_at for s in curiouskiwicat_statuses[2008:2009]]

[datetime.datetime(2018, 1, 5, 2, 7, 10)]

In [42]:
#a tweet at the beginning of 2018
curiouskiwicat_statuses[2008].id

949099924330721280

In [43]:
beginning_of_2018_marker=949099924330721280

In [3]:
def describe_status(s):
    print(f"TWEET by {s.user.screen_name}:")
    print(s.full_text)
    #print(", ".join(dir(s)))
    print(f"is_quote_status: {s.is_quote_status}")
    print(f"in_reply_to_screen_name: {s.in_reply_to_screen_name}")
    print("___\n")
    

In [4]:
#",".join(dir(user_retweets[0]))

So we need to look up the original tweet using the quoted_status_id...

get a set of user statuses that are retweets...

get this user's last 10 statuses

#### Retrieval script to retrieve a set of tweets in my dictionary format and save to a csv

In [ ]:
import time

In [ ]:
#[print(susi.screen_name) for susi in starting_user_set]

In [5]:
users_to_retrieve = starting_user_set

In [6]:
userids_to_retrieve=[utr.screen_name for utr in users_to_retrieve]

In [57]:
def get_users_tweets(user_ids_to_retrieve):
    tweet_rt_pairs = []
    
    rls=api.rate_limit_status()
    #record how much remaining status limit we have
    status_rl_left = rls['resources']['statuses']['/statuses/show/:id']['remaining']
    status_rl_rest_ts = rls['resources']['statuses']['/statuses/show/:id']['reset']
    beginning_of_2018_marker=949099924330721280
    
    #time.sleep((datetime.fromtimestamp(rls['resources']['statuses']['/statuses/show/:id']['reset'])-datetime.now()).seconds)
    for i, u in enumerate(user_ids_to_retrieve):
        

        
        start= time.perf_counter()
        print(f"retrieving tweets for user {u}")
        print(status_rl_left)
    
        try:
            user_statuses=[]
            page_count=0
            for status_page in tweepy.Cursor(
                    api.user_timeline, screen_name=u, include_rts=False, 
                    tweet_mode='extended', count=200,since_id=beginning_of_2018_marker).pages(10):
                page_count += 1
                #print(f'Getting page {page_count} for statuses')
                #print(len(status_page))
                user_statuses.extend(status_page)
            print(f"number of statuses for user: {len(user_statuses)}")
        except tweepy.TweepError as tweep_error:
            if tweep_error.args[0][0]['code']==34:
                    print("somehow browsed through too many timeline pages. quitting")
                    continue
            elif tweep_error.args[0]=='Not authorized.':
                print("couldn't retrieve user tweets. moving on to next user.")
                continue
            

        user_retweets = [s for s in user_statuses if s.is_quote_status==True]
        
        print(f"number of retweets for user: {len(user_retweets)}")

        for s in user_retweets:
            
            if status_rl_left<5:
                #wait a while
                #how long?
                rls=api.rate_limit_status()
                #double check that we are in fact almost out.
                status_rl_left = rls['resources']['statuses']['/statuses/show/:id']['remaining']
                
                if status_rl_left<5:
                    status_rl_rest_ts = rls['resources']['statuses']['/statuses/show/:id']['reset']
                    seconds_to_wait = 1 + (datetime.fromtimestamp(status_rl_rest_ts)-datetime.now()).seconds
                    print(f"status limit almost reached. need to pause for {seconds_to_wait}")
                    time.sleep(seconds_to_wait)
                
            try:
                if hasattr(s,'quoted_status_id'):
                    status_rl_left-=1#subtract from our status limit.
                    original_tweet = api.get_status(s.quoted_status_id,tweet_mode='extended')
                else:
                    print("tweet has not original status Id for some reason. ignoring.")
                    continue
            except tweepy.TweepError as tweep_error:
                try:
                    if tweep_error.args[0][0]['code']==144:
                        print("tweet not found, ignoring")
                        continue
                    elif tweep_error.args[0][0]['code']==179:
                        print("tweet access prohibited, ignoring")
                        continue
                    elif tweep_error.args[0][0]['code']==63:
                        print("user has been suspended")
                        continue
                    elif tweep_error.args[0][0]['code']==34:
                        print("spage error when trying to retrieve RT. continuing")
                        continue
                    else:
                        raise tweep_error
                except Exception as e:
                    print("error while handling exception. passing back exception.")
                    return(e)

            #if the 
            retweet_data = {
                'retweet_text':s.full_text,
                'rt_author':s.user.screen_name,
                'original_text':original_tweet.full_text,
                'original_author':original_tweet.user.screen_name
            }
            #print("retweet of:")
            #describe_status(original_tweet)
            tweet_rt_pairs = tweet_rt_pairs + [retweet_data]

        end= time.perf_counter()
        #iterate through a maximum of 1 user per second.
        duration=end-start
        if(duration<1):
            time.sleep(1-duration)
    return(tweet_rt_pairs)

In [58]:
tweet_rt_pairs = get_users_tweets(userids_to_retrieve)

retrieving tweets for user gammelvindnz
498
number of statuses for user: 1744
number of retweets for user: 135
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
spage error when trying to retrieve RT. continuing
user has been suspended
retrieving tweets for user juliefairey
363
number of statuses for user: 1552
number of retweets for user: 82
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
retrieving tweets for user Jasontripletdad
281
number of statuses for user: 1062
number of retweets for user: 29
tweet access prohibited, ignoring
retrieving tweets for user TheGanttGuy
252
number of statuses for user: 941
number of retweets for user: 88
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
user has been suspended
tweet not found, ignoring
tweet has not original status Id for some reason. ignoring.
retrieving tweets for user d

tweet not found, ignoring
tweet access prohibited, ignoring
retrieving tweets for user camrynpetebrown
514
number of statuses for user: 1927
number of retweets for user: 134
tweet not found, ignoring
tweet access prohibited, ignoring
tweet has not original status Id for some reason. ignoring.
tweet has not original status Id for some reason. ignoring.
retrieving tweets for user brendon_harre
382
number of statuses for user: 958
number of retweets for user: 90
tweet access prohibited, ignoring
spage error when trying to retrieve RT. continuing
spage error when trying to retrieve RT. continuing
retrieving tweets for user Ganeshan
292
number of statuses for user: 832
number of retweets for user: 61
tweet not found, ignoring
user has been suspended
user has been suspended
retrieving tweets for user sharifshameem
231
number of statuses for user: 1025
number of retweets for user: 43
tweet not found, ignoring
tweet has not original status Id for some reason. ignoring.
tweet not found, ignorin

retrieving tweets for user MaramaDavidson
374
number of statuses for user: 1461
number of retweets for user: 130
tweet not found, ignoring
tweet not found, ignoring
retrieving tweets for user JulieAnneGenter
244
number of statuses for user: 1115
number of retweets for user: 56
tweet not found, ignoring
user has been suspended
tweet not found, ignoring
tweet not found, ignoring
retrieving tweets for user Megan_Woods
188
number of statuses for user: 472
number of retweets for user: 48
retrieving tweets for user realestatealarm
140
number of statuses for user: 0
number of retweets for user: 0
retrieving tweets for user propertyhubnz
140
number of statuses for user: 0
number of retweets for user: 0
retrieving tweets for user buildbetternz
140
number of statuses for user: 662
number of retweets for user: 129
tweet not found, ignoring
spage error when trying to retrieve RT. continuing
spage error when trying to retrieve RT. continuing
spage error when trying to retrieve RT. continuing
spage 

number of statuses for user: 1380
number of retweets for user: 343
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet access prohibited, ignoring
tweet access prohibited, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet has not original status Id for some reason. ignoring.
retrieving tweets for user Grant_AAA
362
number of statuses for user: 1764
number of retweets for user: 105
tweet has not original status Id for some reason. ignoring.
retrieving tweets for user OckhamRes_NZ
258
number of statuses for user: 19
number of retweets for user: 5
retrieving tweets for user HUDgovtnz
253
number of statuses for user: 81
number of retweets for user: 8
retrieving tweets for user GreaterAKL
245
number of statuses for user: 1383
number of retweets for user: 258
tweet access prohibited, ignoring
tweet not found, ignoring
tweet has not original status Id for some reason. ignoring.
tweet not found

In [ ]:
# try:
#     original_tweet = api.get_status(s.quoted_status_id,tweet_mode='extended')
# except tweepy.TweepError as tweep_error:
#     print(tweep_error)
#     te=tweep_error

In [59]:
import pandas as pd

tweet_rt_df = pd.DataFrame(tweet_rt_pairs)



In [60]:
tweet_rt_df.sample(5)

,retweet_text,rt_author,original_text,original_author
4402,Translated:\n\n✓ No eastern busway\n✓ No rail ...,HendoWest,I believe Kiwis pay too much tax already. That...,simonjbridges
1168,we can offer a fourth. they still won't like i...,carla_org,CM Glover wants a 2nd opinion from another CEQ...,SantaCruzYIMBY
9522,Lol please don’t tell me this account is about...,Grant_AAA,For those who are confused this is something l...,HeartOfTakapuna
2683,There is an academic paper floating around how...,BenRoss_AKL,A witness to the latest Covid-19 border debacl...,radionz
3783,We're hatching plans to be best dressed for th...,DarbyatCouncil,An example playing out all across the country....,H20FrntDsgn


In [63]:
nrow(tweet_rt_df)

NameError: name 'nrow' is not defined

In [61]:
tweet_rt_df.to_csv("../data/housing_peeps_tweets_since2018.csv")

### Step 2. filter these to a training set

In [ ]:
import pandas as pd
tweet_rt_df = pd.read_csv("../data/housing_peeps_tweets.csv")

In [ ]:
tweet_rt_df

In [64]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

In [65]:
def print_tweetrow_md(tweetrow):
    printmd("**TWEET:**")
    printmd(tweetrow['original_text'])
    printmd("*Retweet:*")
    printmd(tweetrow['retweet_text'])

In [66]:
for i,row in tweet_rt_df.sample(5).iterrows():
    print_tweetrow_md(row)

**TWEET:**

@PaulBMcGill Halfway thru, very nice. Mild mushroom flavour and pleasing consistency a bit like firm tofu or well-cooked squid. I reckon it’d be good in a stir fry with some stronger flavours, but this recipe a good start... https://t.co/SNS3AO3eQE https://t.co/gqZiDfZVl8

*Retweet:*

@PtwoAus https://t.co/TEARNhYuFu https://t.co/vHDcXEzs08

**TWEET:**

...

Disappointing. I'm urging the University to do the right thing here.  It's time to put the name-change idea to one side and instead tap into the huge goodwill students, alumni and the Wellington community have for Vic. Let's work together on the issues that matter.  #StickWithVic https://t.co/e8ZAE0vVQo

**TWEET:**

Hard not to like these pop-up lanes on Tamaki Drive - please value our safety over the storage of vehicles @AklTransport https://t.co/vAksVxbPR3

*Retweet:*

The difference 3 days makes - pop ups gone - shared path full of road signs for vehicles. All hail 6 lanes for motor vehicles and trucks @AklTransport 🤯🤯 https://t.co/iM1sEOcP5i https://t.co/ECSErpQUZi

In [ ]:
housing_tweets = [
    t['original_text'] for t in tweet_rt_df.iterrows()
for i,row in tweet_rt_df.sample(5).iterrows():
    printmd("**TWEET:**")
    printmd(row['original_text'])
    printmd("*Retweet:*")
    printmd(row['retweet_text'])

hmm, most of these tweets aren't related to housing. Maybe that's OK. But what if we applied the same filter we applied to the actual tweets?

#### Matching tweets only

In [67]:
matching_tweet_vector = tweet_rt_df['original_text'].str.contains('(housing|homeless|RMA)')

/Users/benjaminsmith/anaconda/envs/tweetenv/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [68]:
matching_tweets = tweet_rt_df[matching_tweet_vector]

In [70]:
matching_tweets.shape

(532, 4)

In [69]:
for i,row in matching_tweets.sample(5).iterrows():
    print_tweetrow_md(row)

**TWEET:**

Many #LosAngeles motels could be converted to provide short term #housing: https://t.co/isCsz75eZe @KPCC @KPCCrina911 #homelessness #Homeless #news

*Retweet:*

Yes! Along with the PSH ordinance, this is a great streamlining bill to speed up provision of housing for homeless Angelenos https://t.co/0d66mYFigp

**TWEET:**

...

I would like to see my taxpayer $ go toward affordable/permanent supportive housing instead of police. #CareNotCops #PeoplesBudgetLA https://t.co/6rzWq1Dzzx

**TWEET:**

Here are slides from my recent talks, "Housing Element Law for YIMBYs (10 ways to do it better)," https://t.co/7uC8bXBdlM. This thread sums it up. All stuff that can be done administratively under CA law which requires local govts to plan for regional housing need.

*Retweet:*

Some quality content for your Thursday. #YIMBY #ThursdayThoughts 

Thank you @CSElmendorf for sharing your work! https://t.co/U5P3sbtYoz


We can go several directions on this:

1. Filter to users who had a tweet matching our filter. Get all their mutual followers. Check mutual follows' RTs; include their housing tweets. Then repeat the process.
2. Identify people who were retweeted. Include them in our database, looking for their tweets. Repeat the process with them.

I'm going to try the first approach. I think approach (2), simply identifying people whose tweets are topical, might be too broad.

#### Approach 1

In [ ]:
import numpy as np

In [ ]:
housing_tweeters = np.unique(np.concatenate([matching_tweets.rt_author.unique(),(matching_tweets.original_author.unique())]))

In [ ]:
housing_tweeters

In [ ]:
api.get_followers_ids()

In [ ]:
res=api.friends_ids(screen_names=['curiouskiwicat'])

In [ ]:
def get_followers(user_id):
    user_ids = []
    page_count = 0
    for userid in tweepy.Cursor(api.followers_ids, id=user_id, count=5000,include_user_entities=False).pages():
        print(api.rate_limit_status()['resources']['followers'])
        start=time.perf_counter()
        page_count += 1
        print(f'Getting page {page_count} for followers')
        print(userid)
        user_ids.extend(userid)
        end= time.perf_counter()
        #iterate through a maximum of 400 users per second
        duration=end-start
        if(duration<1):
            time.sleep(1-duration)
    return user_ids


def get_friends(user_id):
    user_ids = []
    page_count = 0
    for userid in tweepy.Cursor(api.friends_ids, id=user_id, count=5000,include_user_entities=False).pages():
        print(api.rate_limit_status()['resources']['followers'])
        page_count += 1
        print(userid)
        print(f'Getting page {page_count} for friends')
        user_ids.extend(userid)
    return user_ids

In [ ]:
rls=api.rate_limit_status()

In [ ]:
rls['resources']['followers']

In [ ]:
rls['resources']

In [ ]:
users_of_interest = housing_tweeters
new_users_of_interest=[]
for i,u in enumerate(users_of_interest):
    print(f"{i} of {len(users_of_interest)}")
    #followers = get_followers(u)
    friends = get_friends(u)
    mutuals = set(friends).intersection(followers)
    new_users_of_interest = new_users_of_interest + list(mutuals)

This is not going to work because we get rate limited. I'm only able to do as many as 15 follower and friend listings before I get rate limited.

#### Approach 2

I could instead just look at the second approach - just getting people who were RT'd by the original user. Ideally we look at ALL those who were retweeted, not just the ones talking about housing. However, because we are rate limited, it is probably better to limit to just the retweets about housing.

In [71]:
next_pass_of_users=matching_tweets.original_author.unique()

In [72]:
next_pass_of_users

array(['matpottinger', 'radionz', 'Royal_Greenwich', 'TheMurkyDepths',
       'AbundantHomeLDN', 'PeteApps', 'LabourHousing', 'russellcurtis',
       'GoodwinMJ', 'SkyNews', 'RobertKwolek', 'tomcopley', 'AntBreach',
       'winkybiker', 'BDonline', 'CentreforCities', 'tomravenscroft',
       'geographyjim', 'Apex_airspace', 'Leo_Pollak', 'tperry518',
       'CamdenCouncil', 'wfcouncil', 'AFL_tweet', 'stockwool',
       'terraplanner', 'WalnutParkLA', 'UrbanEnviroCA', 'louismirante',
       'CarterRubin', 'StanleyMJohnson', 'esgarciaa', 'graysonapeters',
       'Planning4LA', 'AyannaPressley', 'beyondchron', 'LeftistConnor',
       'AbundantHousing', 'nkburns3', 'CSElmendorf', 'Up4Growth',
       'AmyDoraMD', 'anthonydedousis', 'SophiaBush', 'SDGaction',
       'LosAngelesFwd', 'AOC', 'SonjaTrauss', 'AbundantHomesMA',
       'Scott_Wiener', 'ahvancouver', 'mlevinreports', 'CurbedLA',
       'BarackObama', 'MikeBoninLA', 'SenGonzalez_33', 'DavidZahniser',
       'DotKohlhaas', 'ShaneDPhi

right, now we iterate through them

In [73]:
user_tweets2 = get_users_tweets(next_pass_of_users)

retrieving tweets for user matpottinger
900
number of statuses for user: 684
number of retweets for user: 18
retrieving tweets for user radionz
882
number of statuses for user: 1712
number of retweets for user: 101
retrieving tweets for user Royal_Greenwich
781
number of statuses for user: 1758
number of retweets for user: 38
retrieving tweets for user TheMurkyDepths
743
number of statuses for user: 1199
number of retweets for user: 231
tweet not found, ignoring
tweet not found, ignoring
spage error when trying to retrieve RT. continuing
spage error when trying to retrieve RT. continuing
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
retrieving tweets for user AbundantHomeLDN
512
number of statuses for user: 973
number of retweets for user: 304
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
spage error when trying to retrieve RT. continuing
tweet not found, ignoring
tweet n

tweet has not original status Id for some reason. ignoring.
tweet has not original status Id for some reason. ignoring.
tweet has not original status Id for some reason. ignoring.
tweet has not original status Id for some reason. ignoring.
retrieving tweets for user nkburns3
560
number of statuses for user: 1756
number of retweets for user: 320
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet has not original status Id for some reason. ignoring.
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet not found, ignoring
tweet has not original status Id for some reason. ignoring.
tweet not found, ignoring
tweet not found, ignoring
tweet has not original status Id for some reason. ignoring.
tweet not found, ignoring
tweet not found, ignoring
tweet access prohibited, ignoring
tweet not found, ignoring
retrieving tweets for user CSElmendorf
243
number of statuses for user: 773
number of retweets for us

In [74]:
tweet_rt_df_l2 = pd.DataFrame(user_tweets)

NameError: name 'user_tweets' is not defined

In [ ]:
matching_tweet_vector = tweet_rt_df['original_text'].str.contains('(housing|homeless|RMA)')
matching_tweets = tweet_rt_df[matching_tweet_vector]

In [ ]:
for i,row in matching_tweets.sample(5).iterrows():
    print_tweetrow_md(row)

In [ ]:
tweet_rt_df = pd.read_csv("../data/housing_peeps_tweets.csv")

def get_housing_tweets(full_tweet_list):
    matching_tweet_vector = full_tweet_list['original_text'].str.contains('(housing|homeless|RMA)')
    matching_tweets = full_tweet_list[matching_tweet_vector]
    return(matching_tweets)



In [ ]:
full_set=pd.concat([get_housing_tweets(tweet_rt_df),get_housing_tweets(tweet_rt_df_l2)]).reset_index(drop=True)


## Save

In [ ]:
full_set.to_csv("../data/housing_peeps_tweets_full.csv")

In [ ]:
full_set

In [ ]:
def convert_row_to_plaintext(row):
    return(row['original_text'] + "\nRT: \n" +  row['retweet_text'] + "\n<|endoftext|>\n")
plaintext_vals=full_set.apply(convert_row_to_plaintext,1)
plain_text_out = "\n".join(plaintext_vals)

In [ ]:
print(plain_text_out)

In [ ]:
raise Exception("should trim out the links, probably")
with open("../data/housing_fulltext_plain.txt",'w') as text_file:
    print(plain_text_out,file=text_file)